In [1]:
import pygmtools as pygm
import numpy as np

In [2]:
arr=np.random.randn(2000,2000)
arr

array([[ 0.05797363,  0.87954661, -0.61191124, ...,  0.5669722 ,
         0.52839373,  0.59275062],
       [ 0.77004291, -0.57310928,  0.4396872 , ..., -0.07555901,
        -0.40941372,  1.99160837],
       [-2.04404998, -0.15981711,  0.81283123, ..., -0.31931728,
         0.81326674,  1.03218471],
       ...,
       [ 1.1174069 ,  0.18813755, -1.65900954, ..., -0.26138342,
        -0.1556713 , -0.58335245],
       [-0.79019561, -1.21512263, -0.01060147, ...,  1.08878954,
        -0.73816048,  2.21783624],
       [ 0.00787274, -0.87846207, -0.47836808, ..., -1.04628122,
         0.7337717 , -0.03565108]])

In [3]:
%timeit y_=pygm.hungarian(arr)

120 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit z=pygm.sinkhorn(arr) 

216 ms ± 4.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
